### Get UniProt Protein Names/Synonyms

In [2]:
import pandas as pd, requests
# IDs = ... whatever the list of your proteins' UniProt IDs are

In [ ]:
def get_short_full_names(names, section_key, section_values):
    '''
    FUNCTION:
    - Get all names from an entry
    
    PARAMS:
    - names (list)
    '''
    # Full name
    if section_key == 'fullName':
        name = section_values['value']
        names.append(name)

    # Short names
    if section_key == 'shortNames':
        for entry in section_values:
            name = entry['value']
            names.append(name)  
            
    # EC Numbers
    if section_key == 'ecNumbers':
         for entry in section_values:
            name = entry['value']
            names.append(name) 
            
    return names




id2syns = dict()

SIZE = str(500)

# For each ID, get the protein names (one protein at a time)
for index, ID in enumerate(IDs):
    print(index+1,'/',len(IDs),end='\r')
    names = list()
    
    # Get data via UniProt API 
    url = 'https://rest.uniprot.org/uniprotkb/search?fields=accession,protein_name,ec&format=json&query='\
           +ID+'&size='+SIZE
    res = requests.get(url).json()
    r = res['results'][0]['proteinDescription']

    
    
    ### Recommended name ###
    try:
        # Find sections
        rec_name_section = r['recommendedName']
        for section_key, section_values in rec_name_section.items():
            
            # Get names
            names = get_short_full_names(names, section_key, section_values)
            
        print(ID, rec_name_section)
        print(names)
    except:
        pass
    
    
    ### Alternative Names ###
    try:
        # Find sections
        alt_name_section = r['alternativeNames']
        for entry in alt_name_section:
            for section_key, section_values in entry.items():
                
                # Get names
                names = get_short_full_names(names, section_key, section_values)
    except:
        pass
    
    
    ### Submitted Names ###
    try:
        # Find sections
        sub_name_section = r['submissionNames']
        for entry in sub_name_section:
            for section_key, section_values in entry.items():

                # Get names
                names = get_short_full_names(names, section_key, section_values)     
    except:
        pass
    
    
    id2syns[ID] = names
    
json.dump(id2syns, open('input/id2names_not_case_varied.json','w'))